In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

from google.colab import drive
drive.mount('/content/drive')

!unzip glove.6B.zip -d /content/
!pip install contractions
import nltk
nltk.download('all')

--2019-06-15 16:52:19--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-06-15 16:52:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-06-15 16:52:19--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

True

In [0]:
# Preprocessing the train_data
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
!pip install -U nltk
import os
import nltk
nltk.download('all')
from nltk.corpus import stopwords

from tqdm import tqdm
import numpy as np
import os
import sys
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


#preprocessing methods
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words


#path of training data
train_path =  '/content/drive/My Drive/ML_Datasets/genData/MTdata'
col_names = ['X.AUTHID','site.content','cSIN_tag','cEXC_tag','cCOM_tag','cRUG_tag','cSOP_tag']
file_names = ['com_mt_essays_16thMay.csv','rug_mt_essays_27thMay.csv','exc_mt_essays_16thMay.csv','sinc_mt_essays_16thMay.csv','sop_mt_essays_16thMay.csv']
f_names = ['rug_mt_essays_27thMay.csv']
for files in os.listdir(train_path):
    if files in f_names:
        reader = csv.DictReader(open(train_path+'/'+files,encoding='latin-1'))
        
        datalist = []
        for raw in reader:
            datalist.append((raw['X.AUTHID'],raw['site.content'],raw['cSIN_tag'],raw['cEXC_tag'],raw['cCOM_tag'],raw['cRUG_tag'],raw['cSOP_tag']))


train_data = np.array(datalist)
data = pd.DataFrame.from_records(datalist, columns=col_names)
print(train_data.shape)

pickle_out = open("train_data_rug.pickle","wb")
pickle.dump(data, pickle_out)
pickle_out.close()


words = []
for texts in data['site.content']:
    text = denoise_text(texts)
    text = replace_contractions(text)
    word = nltk.word_tokenize(text)
    word = normalize(word)
    words.append(word)

words = np.array(words)
print(words.shape)

new_text = []
for i in range(len(words)):
    text = " ".join(str(x) for x in words[i])
    new_text.append(text)

print(len(new_text))

pickle_out = open("text_RUG.pickle","wb")
pickle.dump(new_text, pickle_out)
pickle_out.close()


    

Using TensorFlow backend.


     |████████████████████████████████| 1.5MB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/54/40/b7/c56ad418e6cd4d9e1e594b5e138d1ca6eec11a6ee3d464e5bb
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [0]:
#model training, saving the doc level features
!pip install imblearn
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE
# /content/drive/My Drive/ML_Datasets/genData/text.pickle

#loading the pickled files
pickle_in = open("text_RUG.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("train_data_rug.pickle","rb")
data = pickle.load(pickle_in)

embedding_index = {}
#creating embedding matrix
with open('glove.6B.300d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,'f',sep=' ')
        embedding_index[word] = coefs


#hyperparameters
vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

#tokenized the texts, to form the numerical vector
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)

X = pad_sequences(train_sentences_tokenized, maxlen=max_len)
# print(X.shape)
# print(X)
# print(tokenizer.word_index)

#concatenating the binary labels of all personality traits
tags = ['y','n']
label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN_tag'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC_tag'])
Y_2 = to_categorical(Y_2)
Y_3 = label_enc.transform(data['cCOM_tag'])
Y_3 = to_categorical(Y_3)
Y_4 = label_enc.transform(data['cRUG_tag'])
Y_rug = Y_4
Y_4 = to_categorical(Y_4)
Y_5 = label_enc.transform(data['cSOP_tag'])

Y_5 = to_categorical(Y_5)
Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)
print(Y.shape)
# # for rugggedness only
sm = SMOTE(random_state=4991, n_jobs=8, ratio={1:2500, 0:2500})
new_X,new_Y = sm.fit_sample(X,Y_rug)
print(new_X.shape,new_Y.shape)
new_Y = to_categorical(np.reshape(new_Y,(-1,1)))
# ###############


#data splitting
train_X,test_X,train_Y,test_Y = train_test_split(new_X,new_Y,test_size = 0.1,random_state = 4991)

print('Preparing embedding matrix')
num_words = min(vocab_size,len(word_index))+1
embedding_matrix = np.zeros((num_words,embedding_dim))

for word,i in word_index.items():
    if i > vocab_size:
        continue

    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


embedding_layer = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = max_len,
                            trainable = False)

#creating the model
sequence_input = Input(shape = (max_len,),dtype = 'int32')
embedded_sequences = embedding_layer(sequence_input)
x1 = Conv1D(64,1,activation = 'relu')(embedded_sequences)
# x1 = GlobalMaxPooling1D()(x1)
x2 = Conv1D(64,2,activation = 'relu')(embedded_sequences)
# x2 = GlobalMaxPooling1D()(x2)
x3 = Conv1D(64,3,activation = 'relu')(embedded_sequences)
# x3 = GlobalMaxPooling1D()(x3)
print(x1.shape,x2.shape,x3.shape)
x = concatenate([x1,x2,x3],axis=1)
print(x.shape)
x = Dropout(0.5)(x)
x = Conv1D(128,3,activation = 'relu')(x)
docvec = GlobalMaxPooling1D()(x)
doc_model = Model(sequence_input,docvec)
x = Dropout(0.4)(docvec)
x = Dense(64,activation = 'relu')(x)
x = Dropout(0.5)(x)
pred = Dense(2,activation = 'softmax')(x)
model = Model(sequence_input,pred)
# model.summary()

adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)

model.compile(loss = 'binary_crossentropy',optimizer = adam ,metrics = ['accuracy'])
model.summary()
a,b = 0,2
model.fit(train_X,train_Y[:,a:b],batch_size = batch_size,epochs = 100,validation_data = (test_X,test_Y[:,a:b]))
scores = model.evaluate(test_X, test_Y[:,a:b], verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
plot_model(model, to_file='model.png',show_shapes = True)

model.save('my_model_cRUG_tag')

#saving the doc model(document level features)
doc_model.save('doc_model_cRUG')

pred = model.predict(test_X)
pred = pred.argmax(axis=1)
    
c_matrix = confusion_matrix(test_Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(test_Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(test_Y[:,a:b].argmax(axis=1),pred))

pickle_out = open("tokenizer.pickle","wb")
pickle.dump(tokenizer, pickle_out)
pickle_out.close()


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


(2781, 10)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(5000, 3000) (5000,)
Preparing embedding matrix
Instructions for updating:
Colocations handled automatically by placer.
(?, 3000, 64) (?, 2999, 64) (?, 2998, 64)
(?, 8997, 64)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 3000, 300)    9000300     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 3000, 64)     19264       embedding_1[0][0]                
________________

In [0]:
#concatenating the doc level features of dl-final model to LIWC features
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE

file_names = file_names = ['com_mt_mrs_16thMay.csv','rug_mt_mrs_27thMay.csv','exc_mt_mrs_16thMay.csv','sinc_mt_mrs_16thMay.csv','sop_mt_mrs_16thMay.csv']
#loading the doc model
model = load_model('doc_model_cRUG')

#loading the pickled files
pickle_in = open("text_RUG.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("train_data_rug.pickle","rb")
data = pickle.load(pickle_in)



pickle_in = open("/content/drive/My Drive/ML_Datasets/mrs_column.pickle","rb")
col_names = pickle.load(pickle_in)
col_names = col_names[1:len(col_names)-1]

vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)
X = pad_sequences(train_sentences_tokenized, maxlen=max_len)
doc_vec = model.predict(X)
doc_vec = np.array(doc_vec)
print(doc_vec.shape)

#loading the mairesse features
mrs_features = pd.read_csv('/content/drive/My Drive/ML_Datasets/genData/MTdata/rug_mt_mrs_27thMay.csv',usecols = col_names)
print(mrs_features.shape)
# print(mrs_features.head())
mrs_features = np.array(mrs_features,dtype = 'float64')
print(mrs_features.shape)
#concatenating the LIWC features to doc-level features
doc_vec = np.concatenate((doc_vec,mrs_features),axis = 1)
print(doc_vec.shape)
pickle_out = open("mrs_doc_cRUG.pickle","wb")
pickle.dump(doc_vec, pickle_out)
pickle_out.close()

# col_names = []
# for col in mrs_features.columns:
#     col_names.append(col)

# print(col_names)
# pickle_out = open("mrs_column.pickle","wb")
# pickle.dump(col_names, pickle_out)
# pickle_out.close()


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(2781, 128)
(2781, 70)
(2781, 70)
(2781, 198)


In [0]:
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE

#loading the pickled concatenated doc level features
pickle_in = open("mrs_doc_cRUG.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

#loading the train data
pickle_in = open("train_data_rug.pickle","rb")
data = pickle.load(pickle_in)


vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

tags = ['y','n']
label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN_tag'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC_tag'])
Y_2 = to_categorical(Y_2)
Y_3 = label_enc.transform(data['cCOM_tag'])
Y_3 = to_categorical(Y_3)
Y_4 = label_enc.transform(data['cRUG_tag'])

Y_4 = to_categorical(Y_4)

Y_5 = label_enc.transform(data['cSOP_tag'])

Y_5 = to_categorical(Y_5)

#conactenating the binary labels
Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)
print(Y.shape)

#train and validation split
train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size = 0.1,random_state = 4991)


doc_shape = X.shape[1]

#creatng the model consisting of just the classification layers
inp = Input(shape = (doc_shape,),dtype = 'float32')
x = Dropout(0.4)(inp)
x = Dense(64,activation = 'relu')(x)
x = Dropout(0.5)(x)
pred = Dense(2,activation = 'softmax')(x)
model = Model(inp,pred)

adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)

model.compile(loss = 'binary_crossentropy',optimizer = 'adam' ,metrics = ['accuracy'])
model.summary()
a,b = 6,8 #it decides on which trait we are training our model or testing
model.fit(train_X,train_Y[:,a:b],batch_size = batch_size,epochs = 100,validation_data = (test_X,test_Y[:,a:b]))
scores = model.evaluate(test_X, test_Y[:,a:b], verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#saving the model
model.save('mrs_model_cRUG')
pred = model.predict(test_X)
pred = pred.argmax(axis=1)
    
#printing the classification metrics
c_matrix = confusion_matrix(test_Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(test_Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(test_Y[:,a:b].argmax(axis=1),pred))





(2781, 198)
(2781, 10)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 198)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 198)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                12736     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 130       
Total params: 12,866
Trainable params: 12,866
Non-trainable params: 0
_________________________________________________________________
Train on 2502 samples, validate on 279 samples
Epoch 1/100
2502/2502 [==============================] - 1s 215us/st

In [0]:
# concatenating doc level deatures to LIWC features 
# heldout_data


import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE

file_names = file_names = ['com_mt_mrs_16thMay.csv','rug_mt_mrs_27thMay.csv','exc_mt_mrs_16thMay.csv','sinc_mt_mrs_16thMay.csv','sop_mt_mrs_16thMay.csv']
model = load_model('doc_model_cSOP')

pickle_in = open("/content/drive/My Drive/ML_Datasets/genData/test_text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/content/drive/My Drive/ML_Datasets/genData/test_data.pickle","rb")
data = pickle.load(pickle_in)

pickle_in = open("/content/drive/My Drive/ML_Datasets/mrs_column.pickle","rb")
col_names = pickle.load(pickle_in)
col_names = col_names[1:len(col_names)-1]

vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)
X = pad_sequences(train_sentences_tokenized, maxlen=max_len)
doc_vec = model.predict(X)
doc_vec = np.array(doc_vec)
print(doc_vec.shape)
# /content/drive/My Drive/ML_Datasets/genData/MTdata/sinc_mt_mrs_16thMay.csv
mrs_features = pd.read_csv('/content/drive/My Drive/ML_Datasets/genData/mrs_heldout_data_16thMay.csv',usecols = col_names)
print(mrs_features.shape)
mrs_features = np.array(mrs_features,dtype = 'float64')
print(mrs_features.shape)
doc_vec = np.concatenate((doc_vec,mrs_features),axis = 1)
print(doc_vec.shape)
pickle_out = open("heldout_mrs_doc_cSOP.pickle","wb")
pickle.dump(doc_vec, pickle_out)
pickle_out.close()

# col_names = []
# for col in mrs_features.columns:
#     col_names.append(col)

# print(col_names)
# pickle_out = open("mrs_column.pickle","wb")
# pickle.dump(col_names, pickle_out)
# pickle_out.close()



/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(336, 128)
(336, 70)
(336, 70)
(336, 198)


In [0]:
# concatenating doc level deatures to mairesse features 
# ht_data


import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE

file_names = file_names = ['com_mt_mrs_16thMay.csv','rug_mt_mrs_27thMay.csv','exc_mt_mrs_16thMay.csv','sinc_mt_mrs_16thMay.csv','sop_mt_mrs_16thMay.csv']
model = load_model('doc_model_cSOP')

pickle_in = open("ht_test_text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("ht_test_data.pickle","rb")
data = pickle.load(pickle_in)

pickle_in = open("/content/drive/My Drive/ML_Datasets/mrs_column.pickle","rb")
col_names = pickle.load(pickle_in)
col_names = col_names[1:len(col_names)-1]

vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)
X = pad_sequences(train_sentences_tokenized, maxlen=max_len)
doc_vec = model.predict(X)
doc_vec = np.array(doc_vec)
print(doc_vec.shape)
# /content/drive/My Drive/ML_Datasets/genData/MTdata/sinc_mt_mrs_16thMay.csv
mrs_features = pd.read_csv('/content/drive/My Drive/ML_Datasets/genData/ht4_mrs_liwc_data_12thFeb.csv',usecols = col_names)
print(mrs_features.shape)
mrs_features = np.array(mrs_features,dtype = 'float64')
print(mrs_features.shape)
doc_vec = np.concatenate((doc_vec,mrs_features),axis = 1)
print(doc_vec.shape)
pickle_out = open("ht_mrs_doc_cSOP.pickle","wb")
pickle.dump(doc_vec, pickle_out)
pickle_out.close()

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(500, 128)
(500, 70)
(500, 70)
(500, 198)


In [0]:
# heldout test data prediction(doc_level + LIWC features)

import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle

#loa
pickle_in = open("heldout_mrs_doc_cSOP.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pickle_in = open("/content/drive/My Drive/ML_Datasets/genData/test_data.pickle","rb")
data = pickle.load(pickle_in)
model = load_model('mrs_model_cSOP')

vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

tags = ['y','n']
label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN'])

Y_2 = label_enc.transform(data['cEXC'])

Y_3 = label_enc.transform(data['cCOM'])

Y_4 = label_enc.transform(data['cRUG'])

Y_5 = label_enc.transform(data['cSOP'])


print('\n\nSophistication')

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_5
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))

print('\n\nSincerity')

pickle_in = open("heldout_mrs_doc_cSIN.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)
model = load_model('mrs_model_cSIN')

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_1
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))

print('\n\nExcitement')

model = load_model('mrs_model_cEXC')

pickle_in = open("heldout_mrs_doc_cEXC.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_2
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))


print('\n\nCompetence')
model = load_model('mrs_model_cCOM')

pickle_in = open("heldout_mrs_doc_cCOM.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_3
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))

print('\n\nRuggedness')
model = load_model('mrs_model_cRUG')

pickle_in = open("heldout_mrs_doc_cRUG.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_4
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))





(336, 198)


Sophistication
[[112  30]
 [153  41]]
Accuracy :  0.45535714285714285
              precision    recall  f1-score   support

           0       0.42      0.79      0.55       142
           1       0.58      0.21      0.31       194

    accuracy                           0.46       336
   macro avg       0.50      0.50      0.43       336
weighted avg       0.51      0.46      0.41       336

0.45535714285714285
0.5774647887323944
0.211340206185567
0.30943396226415093


Sincerity
(336, 198)
[[ 54   3]
 [268  11]]
Accuracy :  0.19345238095238096
              precision    recall  f1-score   support

           0       0.17      0.95      0.28        57
           1       0.79      0.04      0.08       279

    accuracy                           0.19       336
   macro avg       0.48      0.49      0.18       336
weighted avg       0.68      0.19      0.11       336

0.19345238095238096
0.7857142857142857
0.03942652329749104
0.0750853242320819


Excitement
(336, 198)
[[ 64 

In [0]:
# ht test data prediction doc_level_features + LIWC features

import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle


pickle_in = open("ht_mrs_doc_cSOP.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pickle_in = open("ht_test_data.pickle","rb")
data = pickle.load(pickle_in)
model = load_model('mrs_model_cSOP')

vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

tags = ['y','n']
label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN'])

Y_2 = label_enc.transform(data['cEXC'])

Y_3 = label_enc.transform(data['cCOM'])

Y_4 = label_enc.transform(data['cRUG'])

Y_5 = label_enc.transform(data['cSOP'])


print('\n\nSophistication')

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_5
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))

print('\n\nSincerity')

pickle_in = open("ht_mrs_doc_cSIN.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)
model = load_model('mrs_model_cSIN')

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_1
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))

print('\n\nExcitement')

model = load_model('mrs_model_cEXC')

pickle_in = open("ht_mrs_doc_cEXC.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_2
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))


print('\n\nCompetence')
model = load_model('mrs_model_cCOM')

pickle_in = open("ht_mrs_doc_cCOM.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_3
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))

print('\n\nRuggedness')
model = load_model('mrs_model_cRUG')

pickle_in = open("ht_mrs_doc_cRUG.pickle","rb")
X = pickle.load(pickle_in)
print(X.shape)

pred = model.predict(X)
pred = pred.argmax(axis=1)
Y = Y_4
c_matrix = confusion_matrix(Y,pred)
print(c_matrix)
accuracy = accuracy_score(Y,pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y,pred))
print(accuracy)
print(precision_score(Y,pred))
print(recall_score(Y,pred))
print(f1_score(Y,pred))





(500, 198)


Sophistication
[[178  46]
 [215  61]]
Accuracy :  0.478
              precision    recall  f1-score   support

           0       0.45      0.79      0.58       224
           1       0.57      0.22      0.32       276

    accuracy                           0.48       500
   macro avg       0.51      0.51      0.45       500
weighted avg       0.52      0.48      0.43       500

0.478
0.5700934579439252
0.2210144927536232
0.31853785900783294


Sincerity


FileNotFoundError: ignored

In [0]:
#splitting the texts into sentences
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import csv
import pandas as pd
import pickle

stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
remove_punctuation_map = dict((ord(char), ' ') for char in string.punctuation)
integer_chars = [str(digit) for digit in list(range(10))]
remove_integer_map = dict((ord(char), None) for char in integer_chars)

def sentsplit(docs):
    '''
    Splits each document of a document list(collection) into sentences and returns the split list
    '''
    return [nltk.sent_tokenize(doc) for doc in docs]

def stem_tokens(tokens):
    '''
    Transforms a list of words into a list of stemmed words
    '''
    return [stemmer.stem(item) for item in tokens]

def lemm_tokens(tokens):
    '''
    Transforms a list of words into a list of lemmatised words
    '''
    return [lemmatizer.lemmatize(item) for item in tokens]

def filtersent(text, remove_stopwords=True, stem=False, lemm=True, avoid_single_char=True):
    '''
    Filters a sentence according to requirements
    '''
    # remove puctuations and lower the case
    simpletext = text.lower().translate(remove_punctuation_map).translate(remove_integer_map)
    # tokenize
    words = nltk.word_tokenize(simpletext)
    # remove stop words
    if remove_stopwords:
        words = [w for w in words if w not in stopwords.words('english')]
    # lemmatize them
    if lemm:
        words = lemm_tokens(words)
    # stem them
    if stem:
        words = stem_tokens(words)
    # avoid single character words
    if avoid_single_char:
        words = [w for w in words if w not in string.ascii_lowercase]
    # detokenise the sentence
    return nltk.tokenize.treebank.TreebankWordDetokenizer().detokenize(words)

path = '/content/drive/My Drive/ML_Datasets/toAnnotateDataPoints7thJune.csv'

reader = csv.DictReader(open(path,encoding='latin-1'))
        
datalist = []
col_names = ['X.AUTHID','site.content','cSIN_label_fin','cEXC_label_fin','cCOM_label_fin','cRUG_label_fin','cSOP_label_fin']
for raw in reader:
    datalist.append((raw['X.AUTHID'],raw['site.content'],raw['cSIN_label_fin'],raw['cEXC_label_fin'],raw['cCOM_label_fin'],raw['cRUG_label_fin'],raw['cSOP_label_fin']))
data = pd.DataFrame.from_records(datalist, columns=col_names)  
text = data['site.content']

for t in text:
  print(t)
  
print('------------------------------------------------------------------------------------\n\n')
sentences = sentsplit(text)
for sent in sentences:
  print(sent)

X = []
auth_id = {}
for (doc,a) in zip(sentences,data['X.AUTHID']):
    s = []
    for sent in doc:
        x = filtersent(sent)
        X.append(x)
        s.append(x)
    
    auth_id[a] = s

# X = np.array(X)
# X = np.reshape(X,(-1,1))
# print(X.shape)
pickle_out = open("annote_data.pickle","wb")
pickle.dump(data, pickle_out)
pickle_out.close()
pickle_out = open("annote_sent.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()




b'(1);Are you tired of those pesky jagged diagonal lines and unsmooth curves in AutoCAD? AutoCAD 2017 to the rescue! Yup, its true: The appearance, selection, and performance of 2D geometry in this release arepretty great. Dont believe me? Ive got the before and after photos:;And unlike those diet pill infomercials, these results ;, in fact, typical. Suuuuuure, youre probably saying, but how do I make it happen? Heres how:;First up, to e this beautiful performance increase, youll need a DirectX 11-capable graphics card and your hardware acceleration turned ;. But, dont worry, we make it easy for you to make the adjustment. Just click on the Graphics Performance dialog box (in the status bar) and youll see the high quality geometry option:;This all sounds great, but Im not sure if I have the right hardware, Lynn! OK, dont worry, weve got you covered regardless: If your hardware doesnt support high quality geometry (i.e., isnt DirectX 11-capable), just try using the smooth line display o

In [0]:
from itertools import islice
path = '/content/drive/My Drive/ML_Datasets/output.csv'
reader = csv.DictReader(open(path,encoding='latin-1'))
datalist = []
col_names = ['Text','SIN','EXC','COM','RUG','SOP']
for raw in reader:
    datalist.append((raw['Text '],raw[' SIN '],raw[' EXC '],raw[' COM '],raw[' RUG '],raw[' SOP ']))
data = pd.DataFrame.from_records(datalist, columns=col_names)  
# print(data)
with open('output_final.csv','w') as f:
    f.write('AUTH_ID,Text,SIN,EXC,COM,RUG,SOP \n')
c = 0
x = 0
for doc,sentences in auth_id.items():
    c += len(sentences)
    z = 0
    print(doc,x)
    for (sent,row) in zip(sentences,islice(data.itertuples(index=True, name='Pandas'),x,None)):
        
        x+=1
        print(str(x),sent,getattr(row, "Text"))
        
        with open('output_final.csv','a') as f:
            f.write(str(doc)+','+getattr(row, "Text")+','+getattr(row, "SIN")+','+getattr(row, "EXC")+','+getattr(row, "COM")+','+getattr(row, "RUG")+','+getattr(row, "SOP")+'\n')
            




b'http://blogs.autodesk.com/autocad/graphics-enhancements-part-1-tuesday-tips-lynn/' 0
1 tired pesky jagged diagonal line unsmooth curve autocad tired pesky jagged diagonal line unsmooth curve autocad 
2 autocad rescue autocad rescue 
3 yup true appearance selection performance geometry release arepretty great yup true appearance selection performance geometry release arepretty great 
4 dont believe dont believe 
5 ive got photo unlike diet pill infomercial result fact typical ive got photo unlike diet pill infomercial result fact typical 
6 suuuuuure youre probably saying make happen suuuuuure youre probably saying make happen 
7 here first beautiful performance increase youll need directx capable graphic card hardware acceleration turned here first beautiful performance increase youll need directx capable graphic card hardware acceleration turned 
8 dont worry make easy make adjustment dont worry make easy make adjustment 
9 click graphic performance dialog box status bar youll see h

In [0]:
#annotation sentences prediction

import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle


vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000
# content/drive/My Drive/ML_Datasets/genData/test_data.pickle
pickle_in = open("annote_sent.pickle","rb")
text = pickle.load(pickle_in)

pickle_in = open("annote_data.pickle","rb")
data = pickle.load(pickle_in)

pickle_in = open("tokenizer.pickle","rb")
tokenizer = pickle.load(pickle_in)
# tokenizer = Tokenizer(num_words = vocab_size)
# tokenizer.fit_on_texts(X)
# word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(text)

X = pad_sequences(train_sentences_tokenized, maxlen=max_len)

tags = ['y','n']

label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN_label_fin'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC_label_fin'])
Y_2 = to_categorical(Y_2)

Y_3 = label_enc.transform(data['cCOM_label_fin'])
Y_3 = to_categorical(Y_3)

Y_4 = label_enc.transform(data['cRUG_label_fin'])
Y_4 = to_categorical(Y_4)

Y_5 = label_enc.transform(data['cSOP_label_fin'])
Y_5 = to_categorical(Y_5)

Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)
print(Y.shape)

model1 = load_model('DL_Final_SIN2')
model2 = load_model('DL_Final_EXC2')
model3 = load_model('DL_Final_COM2')
model4 = load_model('DL_Final_RUG')
model5 = load_model('DL_Final_SOP2')


Y1 = model1.predict(X)
Y2 = model2.predict(X)
Y3 = model3.predict(X)
Y4 = model4.predict(X)
Y5 = model5.predict(X)
Y1 = Y1.argmax(axis=1) 
Y2 = Y2.argmax(axis=1) 
Y3 = Y3.argmax(axis=1) 
Y4 = Y4.argmax(axis=1)
Y5 = Y5.argmax(axis=1) 
# np.save('SIN_pred',Y1.argmax(axis=1))
# np.save('EXC_pred',Y2.argmax(axis=1))
# np.save('COM_pred',Y3.argmax(axis=1))
# np.save('RUG_pred',Y4.argmax(axis=1))
# np.save('SOP_pred',Y5.argmax(axis=1))


with open('output.csv','w') as f:
    f.write('Text , SIN , EXC , COM , RUG , SOP \n')
for i in range(len(text)):
    with open('output.csv','a') as f:
        f.write(text[i]+' , '+str(Y1[i])+' , '+str(Y2[i])+' , '+str(Y3[i])+' , '+str(Y4[i])+' , '+str(Y5[i])+' \n')


from itertools import islice
path = '/content/drive/My Drive/ML_Datasets/output.csv'
reader = csv.DictReader(open(path,encoding='latin-1'))
datalist = []
col_names = ['Text','SIN','EXC','COM','RUG','SOP']
for raw in reader:
    datalist.append((raw['Text '],raw[' SIN '],raw[' EXC '],raw[' COM '],raw[' RUG '],raw[' SOP ']))
data = pd.DataFrame.from_records(datalist, columns=col_names)  
# print(data)
with open('output_final.csv','w') as f:
    f.write('AUTH_ID,Text,SIN,EXC,COM,RUG,SOP \n')
c = 0
x = 0
for doc,sentences in auth_id.items():
    c += len(sentences)
    z = 0
    print(doc,x)
    for (sent,row) in zip(sentences,islice(data.itertuples(index=True, name='Pandas'),x,None)):
        
        x+=1
        print(str(x),sent,getattr(row, "Text"))
        
        with open('output_final.csv','a') as f:
            f.write(str(doc)+','+getattr(row, "Text")+','+getattr(row, "SIN")+','+getattr(row, "EXC")+','+getattr(row, "COM")+','+getattr(row, "RUG")+','+getattr(row, "SOP")+'\n')
            






(202, 10)


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
# import numpy as np
# import csv

# Y1 = np.load('SIN_pred.npy')
# Y2 = np.load('EXC_pred.npy')
# Y3 = np.load('COM_pred.npy')
# Y4 = np.load('RUG_pred.npy')
# Y5 = np.load('SOP_pred.npy')
